# Import

In [1]:
import psycopg2
import pandas as pd

from IPython.display import display
pd.set_option('display.max_colwidth', 70)

# Connect to database

In [22]:
# Replace these values with your database information
host = "rain.db.elephantsql.com"
user = "mlzwagix"
password = "loYwGChAu7g3DJU18WQCnWPSUgZvelbh"
port = "5432"


# Establish a connection
conn = psycopg2.connect(user=user, password=password, host=host, port=port)
curr = conn.cursor()

# Execute Query

## Create a function

In [23]:
def execute_query(connection, cursor, query):
    try:
        # Execute the query
        curr.execute(query)
        
        # Fetch all results into a list of tuples
        results = cursor.fetchall()
        
        # Get the column names from the cursor description
        column_names = [desc[0] for desc in cursor.description]
        
        # Create a DataFrame from the results and column names
        df = pd.DataFrame(results, columns=column_names)
        
        # Print the DataFrame
        display(df)
    except Exception as error:
        print(error)
    finally:
        conn.commit()

### Overview

Since the data are spread across multiple tables we need to use multiple joins to put them into more readable format.

In [4]:
execute_query(conn, curr, """
SELECT 
    "public"."user"."name" AS username,
    "public"."agent_corporate"."name" AS agent_name,
    property.listing_description,
    property.property_url
FROM 
    "public"."property_listing" AS property
JOIN
    "public"."user"
ON 
    property.user_id = "public"."user"."id"
JOIN
    "public"."agent_corporate"
ON 
    property.agent_id = "public"."agent_corporate"."id"
LIMIT 100
""")

,username,agent_name,listing_description,property_url
0,Hanni,Era Sky Harapan Indah,Rumah Untuk Renovasi di Harapan Indah Cocok Untuk Investasi,https://www.rumah123.com/properti/bekasi/hos15487164/
1,susi ERASKY,Era Sky Harapan Indah,cepat rumah 2 lantai termurah di bulevar hijau harapan indah Bekasi,https://www.rumah123.com/properti/bekasi/hos11582175/
2,Mylana RWSB,RAY WHITE SUMMARECON BEKASI,Rumah Cantik Furnish Siap Huni Taman Puspa Harapan Indah Bekasi,https://www.rumah123.com/properti/bekasi/hos15738122/
3,Andreas Hartoyo,Ray White Harapan Indah,"Rumah Minimalis Full Renov di Cluster Arana, Harapan Indah",https://www.rumah123.com/properti/bekasi/hos14410732/
4,Ernani,RE/MAX Premier,Rumah 3 Lantai Opsi Attic 3kt Cluster Viola Scg Summarecon Crown G...,https://www.rumah123.com/properti/bekasi/hos15505082/
...,...,...,...,...
95,Mentari Pashouses,pashouses.id,Rumah 7 Menit Ke Plaza Metropolitan Siap Huni Bisa Kpr J18168,https://www.rumah123.com/properti/bekasi/hos15150583/
96,Mentari Pashouses,pashouses.id,Rumah Dijual Di Bekasi Siap Huni Bisa Kpr J 16897,https://www.rumah123.com/properti/bekasi/hos15150691/
97,Olivia Esther,pashouses.id,Rumah Dijual Di Grand Wisata Bekasi Siap Huni Bisa Kpr J17176,https://www.rumah123.com/properti/bekasi/hos15250121/
98,Mentari Pashouses,pashouses.id,Rumah Minimalis 2 Lantai Siap Huni di Bekasi Bisa Kpr J17207,https://www.rumah123.com/properti/bekasi/hos15096350/


### How many unique rows for each table are there?

In [5]:
execute_query(conn, curr, """
SELECT
    'public.user' AS table_name,
    COUNT(*) AS row_count
FROM
    "public"."user"

UNION

SELECT
    'public.agent_corporate' AS table_name,
    COUNT(*) AS row_count
FROM
    "public"."agent_corporate"

UNION

SELECT
    'public.property_listing' AS table_name,
    COUNT(*) AS row_count
FROM
    "public"."property_listing"
    
UNION

SELECT
    'public.location' AS table_name,
    COUNT(*) AS row_count
FROM
    "public"."location"

UNION

SELECT
    'public.property_type' AS table_name,
    COUNT(*) AS row_count
FROM
    "public"."property_type"
    
UNION

SELECT
    'public.listing_type' AS table_name,
    COUNT(*) AS row_count
FROM
    "public"."listing_type";
""")


,table_name,row_count
0,public.listing_type,3
1,public.location,86
2,public.property_listing,79192
3,public.user,4313
4,public.agent_corporate,877
5,public.property_type,7


## Question answering

### Give me list of 10 houses with my specified criteria

- under one billion rupiah
- 3 or more beds and bathrooms
- one garage
- located on tambun_selatan
- sorted by the cheapest price while maximizing the number of bedrooms.

In [42]:
execute_query(conn, curr, """
SELECT
    property.id,
    property.price AS price_in_million_idr,
    property.bedroom_count,
    property.bathroom_count,
    property.garage_count,
    property.listing_description,
    property."LT",
    property."LB",
    location."name" AS location_name
FROM 
    "public"."property_listing" property
JOIN
    "public"."location" location
ON 
    property.location_id = location.id
WHERE
    bedroom_count >= 3 
    AND bathroom_count >= 3
    AND garage_count = 1
    AND price <= 1000 
    AND location_id = (SELECT id FROM "public"."location" WHERE name = 'Tambun Selatan, Bekasi')
    AND property_type_id = (SELECT id FROM "property_type" WHERE "name" = 'Rumah')
    ORDER BY price ASC,  bedroom_count DESC
    LIMIT 10;

""")

,id,price_in_million_idr,bedroom_count,bathroom_count,garage_count,listing_description,LT,LB,location_name
0,7349,500,5,3,1,Rumah Hook Griya Asri 2 Siap Huni Strategis Akses Tol Baru,132,250,"Tambun Selatan, Bekasi"
1,9491,550,3,3,1,"Dijual murah, Rumah 120m² di Taman Tridaya Indah Tambun Bekasi",120,100,"Tambun Selatan, Bekasi"
2,4261,650,5,3,1,Dijual Rumah Murah Dan Nyaman di Villa Bekasi Indah Daerah Tambun ...,110,110,"Tambun Selatan, Bekasi"
3,18314,650,5,3,1,Jual rumah 2 lantai super murah di tambun selatan,132,250,"Tambun Selatan, Bekasi"
4,879,650,4,4,1,Rumah murah minimalis siap huni,120,160,"Tambun Selatan, Bekasi"
5,18932,650,4,4,1,Rumah cantik minimalis siap huni,120,160,"Tambun Selatan, Bekasi"
6,15871,750,8,8,1,"Dijual Cepat Kontrakan 8 Pintu Siap Huni di Tambun, Bekasi",270,185,"Tambun Selatan, Bekasi"
7,15870,750,8,8,1,"Dijual Cepat Kontrakan 8 Pintu Siap Huni di Tambun, Bekasi",270,185,"Tambun Selatan, Bekasi"
8,15260,825,3,3,1,Rumah di sumber Jaya tambun kab bekasi depan lapangan dan masjid,135,135,"Tambun Selatan, Bekasi"
9,4255,850,4,3,1,Dijual rumah siap huni di daerah sekitar Tambun Selatan Bekasi,136,145,"Tambun Selatan, Bekasi"


### Where is the cheapest location to buy land property?

The cheapest location to by land property can be defined by just sorting the data by price and limit to 100 rows to get top 100 cheapest land. But question is how do we get the location We can aggregate using count So from those 100 rows we aggregate by location sorted the location by the highest count.

In [132]:
execute_query(conn, curr, """
SELECT
    (SELECT name FROM "public"."location" WHERE id = location_id),
    COUNT(*)
FROM
    (
    SELECT
    *
    FROM 
        "public"."property_listing"
    WHERE
        property_type_id = (SELECT id FROM property_type WHERE name = 'Tanah')
    ORDER BY 
        price ASC
    LIMIT 100
    ) as subquery
GROUP BY
    subquery.location_id
ORDER BY
    count DESC
LIMIT 5;
""")

,name,count
0,"Cikarang, Bekasi",23
1,"Tambun Selatan, Bekasi",7
2,"Jatisampurna, Bekasi",6
3,"Bekasi Timur, Bekasi",6
4,"Cibitung, Bekasi",6


As we can see most of it come from cikarang, bekasi